In [38]:
from sklearn.neural_network import MLPClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay
import numpy as np
from sklearn import tree
from imblearn.pipeline import Pipeline
pd.options.display.max_rows = 4000
%load_ext autoreload
%autoreload 2





The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
from PreproccessPipeline import *


data = pd.read_csv('data/adult.csv')
data["class"] = data["class"].str.strip().str.lower() == "<=50k"

cat = ['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country']

continuous_features = [
    'age',
    'fnlwgt',
    'capital-gain',
    'capital-loss',
    'hours-per-week',
    'education-num'
]

pipeline = PreproccessPipeline(continuous=continuous_features,
                    categories=cat, className="class")


# print(data.head())
# data = pipeline.labelDecode(data)
data.head()


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,True
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,True
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,True
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,True
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,True



Splitting data into training / test allows me to test the accuracy of my model on unseend data.

this split is random however to mainatin class raitios i have used stratisfied sampling.

random state = 1 allows randomisation to remain deterministic, this ensures that test data dos not bleed into the training data

In [40]:
X = data.drop(['class'], axis=1)
y = data['class']
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, stratify=y, random_state=1)
# print(X_train.shape, y_train.shape)


In [41]:
import pickle

filename = "models/adult-nn-pipe.pickle"
clf = pickle.load(open(filename, "rb"))

 

In [42]:
import dice_ml
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


# Dataset for training an ML model
d = dice_ml.Data(dataframe=data,
                 categorical_features=cat,
                 continuous_features=continuous_features,
                 outcome_name='class')

# Pre-trained ML model
m = dice_ml.Model(model=clf, backend="sklearn", model_type='classifier')
# DiCE explanation instance
exp = dice_ml.Dice(d, m, method="genetic",)


In [43]:
import json
import csv

counterFactuals = []
percentage_to_sample = 0.02
# percentage_to_sample = percentage_to_sample * 0.1

f = open('counterfactuals/adult-1.csv', 'w')
first = True
writer = csv.writer(f)
result = exp.generate_counterfactuals(
    X.sample(frac=percentage_to_sample,random_state=1),
    total_CFs=1, 
    # desired_class='>40k',
    # proximity_weight=0.8,
    # sparsity_weight=0.2, 
    # diversity_weight=0.6, 
    # categorical_penalty=0.1,
    # stopping_threshold=0.4,
    # verbose=True
)
result = json.loads(result.to_json())
cfList = result["cfs_list"]
cfList = flattenArray(cfList)
# cfList = lookupClassLabel(cfList, classCodes)
if(first):
    print(result['feature_names_including_target'])
    writer.writerow(result['feature_names_including_target'])
    first = False
writer.writerows(cfList)




  0%|          | 0/651 [00:00<?, ?it/s]/Users/alexvarden/.pyenv/versions/3.9.6/lib/python3.9/site-packages/dice_ml/explainer_interfaces/dice_genetic.py:286: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  query_instance_df_dummies[col] = 0
/Users/alexvarden/.pyenv/versions/3.9.6/lib/python3.9/site-packages/dice_ml/explainer_interfaces/dice_genetic.py:286: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  query_instance_df_dummies[col] = 0
/Users/alexvarden/.pyenv/versions/3.9.6/lib/python3.9/site-packages/dice_ml/explainer_interfaces/

['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'class']


Validate Cfs against model, should get 100% accruracy as these are informed by model

In [44]:
from sklearn.preprocessing import LabelBinarizer


cfdata = pd.read_csv('random-pertuabtions/dry-been-100.csv')
# print(cfdata.head())

X = cfdata.drop(['class'], axis=1)
y = cfdata['class']


y_score = clf.predict_proba(X)

label_binarizer = LabelBinarizer().fit(y)
y_onehot_test = label_binarizer.transform(y)
print(y_onehot_test.shape)  # (n_samples, n_classes)

for class_of_interest in label_binarizer.classes_:
    class_id = np.flatnonzero(label_binarizer.classes_ == class_of_interest)[0]

    RocCurveDisplay.from_predictions(
        y_onehot_test[:, class_id],
        y_score[:, class_id],
        name=f"{class_of_interest} vs the rest",
        color="darkorange",
    )
    plt.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")
    plt.axis("square")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(f"{class_of_interest} vs the rest")
    plt.legend()
    plt.show()


KeyError: "None of [Index(['workclass', 'education', 'marital-status', 'occupation',\n       'relationship', 'race', 'sex', 'native-country'],\n      dtype='object')] are in the [columns]"

In [ ]:
import random



samples = 1000
rows = {}

headings = list(data.columns.values)



for columnName in headings:
    rows[columnName] = []


for _ in range(samples):


    for columnName in headings:
        # print(columnName, data[columnName].dtype.name)
       
        if (data[columnName].dtype.name == 'category'):
            value = None
        elif (data[columnName].dtype.name == 'float64' ):
            value = random.uniform(
                data[columnName].min(), data[columnName].max())
        elif (data[columnName].dtype.name == 'int64'):
            value = random.uniform(
                data[columnName].min(), data[columnName].max())
        rows[columnName].append(value)

randomData = pd.DataFrame(data=rows, columns=data.columns)

X = randomData.drop(['class'], axis=1) 




randomData['class'] = clf.predict(X)

print(randomData.groupby(['class']).size())



randomData.to_csv('random-pertuabtions/dry-been-100.csv',index=False)


class
BARBUNYA      2
BOMBAY      521
CALI        115
DERMASON      2
HOROZ       357
SEKER         3
dtype: int64


In [ ]:
clf.predict(X)


array(['BOMBAY', 'BOMBAY', 'BOMBAY', 'HOROZ', 'BOMBAY', 'HOROZ', 'BOMBAY',
       'HOROZ', 'HOROZ', 'BOMBAY', 'CALI', 'HOROZ', 'CALI', 'BOMBAY',
       'CALI', 'BOMBAY', 'BOMBAY', 'BOMBAY', 'BOMBAY', 'BOMBAY', 'CALI',
       'BOMBAY', 'BOMBAY', 'BOMBAY', 'BOMBAY', 'BOMBAY', 'BOMBAY',
       'HOROZ', 'BOMBAY', 'DERMASON', 'BOMBAY', 'BOMBAY', 'BOMBAY',
       'BOMBAY', 'BOMBAY', 'HOROZ', 'HOROZ', 'HOROZ', 'CALI', 'HOROZ',
       'CALI', 'CALI', 'BOMBAY', 'HOROZ', 'HOROZ', 'HOROZ', 'HOROZ',
       'HOROZ', 'BOMBAY', 'BOMBAY', 'CALI', 'CALI', 'HOROZ', 'BOMBAY',
       'BOMBAY', 'HOROZ', 'CALI', 'HOROZ', 'HOROZ', 'BOMBAY', 'BOMBAY',
       'HOROZ', 'BOMBAY', 'BOMBAY', 'HOROZ', 'HOROZ', 'BOMBAY', 'HOROZ',
       'BOMBAY', 'CALI', 'HOROZ', 'BOMBAY', 'BOMBAY', 'CALI', 'BOMBAY',
       'HOROZ', 'BOMBAY', 'HOROZ', 'BOMBAY', 'BOMBAY', 'BOMBAY', 'HOROZ',
       'BOMBAY', 'HOROZ', 'HOROZ', 'BOMBAY', 'BOMBAY', 'CALI', 'HOROZ',
       'BOMBAY', 'BOMBAY', 'CALI', 'BOMBAY', 'HOROZ', 'BOMBAY', 'CALI',
 